In [1]:
import numpy as np
import glob
import scipy.io as sio
import torch
from torch import nn
import csv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import yaml
import random
from evaluate import error
from mmfi import make_dataset, make_dataloader

### Loda Data

In [2]:
dir = 'd:\\Data\\MMFi_Dataset\\E01\\S02\\A01\\depth\\frame007.png'
_mod, _frame = os.path.split(dir)
print(_mod, _frame)

d:\Data\MMFi_Dataset\E01\S02\A01\depth frame007.png


In [2]:
dataset_root = 'd:\Data\My_MMFi_Data\MMFi_Dataset'
# xian zai shi yong de shi Radar_Fused
with open('config_copy.yaml', 'r') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

train_dataset, val_dataset = make_dataset(dataset_root, config)

S02 ['A01', 'A02', 'A03', 'A04', 'A06', 'A08', 'A09', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A26']
S03 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A17', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S05 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A19', 'A20', 'A21', 'A24', 'A25']
S06 ['A01', 'A03', 'A04', 'A06', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S08 ['A01', 'A02', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A13', 'A14', 'A17', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S09 ['A01', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A17', 'A18', 'A20', 'A21', 'A24', 'A25', 'A27']
S11 ['A01', 'A02', 'A03', 'A05', 'A06', 'A08', 'A10', 'A11', 'A12', 'A13', 'A16', 'A17', 'A18', 'A19', 'A21', 'A23',

KeyboardInterrupt: 

In [3]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.

    dict_keys(['modality', 'scene', 'subject', 'action', 'idx', 'output', 
    'input_rgb', 'input_depth', 'input_lidar', 'input_mmwave'])
    '''
    ## get sequence lengths
    # for t in batch:
    #     print(t.keys())
    #     print(a)
    # #     # print(t[0].shape,t[1].shape)
    kpts = []
    [kpts.append(np.array(t['output'])) for t in batch]
    kpts = torch.FloatTensor(np.array(kpts))

    lengths = torch.tensor([t['input_wifi-csi'].shape[0] for t in batch ])

    # # rgb
    # rgb_data = np.array([(t['input_rgb']) for t in batch ])
    # rgb_data = torch.FloatTensor(rgb_data).permute(0,3,1,2)

    # depth
    # depth_data = np.array([(t['input_depth']) for t in batch ])
    # depth_data = torch.FloatTensor(depth_data).permute(0,3,1,2)

    # # mmwave
    # ## padd
    # mmwave_data = [torch.Tensor(t['input_mmwave']) for t in batch ]
    # mmwave_data = torch.nn.utils.rnn.pad_sequence(mmwave_data)
    # ## compute mask
    # mmwave_data = mmwave_data.permute(1,0,2)

    # # lidar
    # ## padd
    # lidar_data = [torch.Tensor(t['input_lidar']) for t in batch ]
    # lidar_data = torch.nn.utils.rnn.pad_sequence(lidar_data)
    # ## compute mask
    # lidar_data = lidar_data.permute(1,0,2)

    # wifi-csi
    wifi_data = np.array([(t['input_wifi-csi']) for t in batch ])
    wifi_data = torch.FloatTensor(wifi_data)
    # return rgb_data, depth_data, lidar_data, mmwave_data, wifi_data, kpts, lengths
    return wifi_data, kpts

In [4]:
rng_generator = torch.manual_seed(config['init_rand_seed'])
train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)

[True, True, True, True, True]
[False, True, False, True, True]
[True, True, True, True, False]
[False, True, True, True, False]
[True, True, True, True, True]
[True, False, True, True, True]
[True, True, True, True, True]
[False, True, True, True, True]
[True, True, True, True, False]
[True, True, True, False, False]

In [5]:
for i, data in enumerate(train_loader):
    # rgb_data, depth_data, lidar_data, mmwave_data, wifi_data, kpts, lengths = data
    # print(rgb_data[0].shape, depth_data[0].shape, lidar_data[0].shape, mmwave_data[0].shape, wifi_data[0].shape,kpts.shape, lengths.shape)
    # print(rgb_data.shape, depth_data.shape, lidar_data.shape, mmwave_data.shape, wifi_data.shape, kpts.shape, lengths.shape)
    wifi_data, label = data
    print(wifi_data.shape, label.shape)
    break

[True, True, True, True, True]
[False, True, False, True, True]
[True, True, True, True, False]
[False, True, True, True, False]
[True, True, True, True, True]
[True, False, True, True, True]
[True, True, True, True, True]
[False, True, True, True, True]
[True, True, True, True, False]
[True, True, True, False, False]
-----------
[True, True, True, True, True]
[False, True, False, True, True]
[True, True, True, True, False]
[False, True, True, True, False]
[True, True, True, True, True]
[True, False, True, True, True]
[True, True, True, True, True]
[False, True, True, True, True]
[True, True, True, True, False]
[True, True, True, False, False]
-----------


KeyboardInterrupt: 

In [16]:
for i in range(len(wifi_data)):
    print([np.isinf(wifi_data[i]).any(), np.isnan(wifi_data[i]).any()])

[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.uint8), tensor(0, dtype=torch.uint8)]
[tensor(0, dtype=torch.u

### Model

In [21]:
# from depth_benchmark.depth_ResNet18 import *
from CSI_benchmark.models.mynetwork import posenet

model = posenet()

# x = torch.randn(32, 100, 5, 3)
# all_domain_kpts, all_domain_outputs = model(x)
# print(all_domain_kpts[0].shape, all_domain_outputs[1][4].shape)

c:\Users\Chen_Xinyan\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Chen_Xinyan\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import sys
sys.path.insert(0, './CSI_benchmark')
csi_model = torch.load('CSI_benchmark/protocol3_random_1.pkl')
# csi_extractor = csi_feature_extractor(csi_model)
# csi_extractor.eval()

### training 

In [22]:
def test(model, tensor_loader, criterion1, criterion2, device):
    model.eval()
    test_mpjpe = 0
    test_pampjpe = 0
    test_mse = 0
    for data in tqdm(tensor_loader):
        depth_data, kpts = data
        inputs = depth_data.to(device).unsqueeze(1)
        kpts.to(device)
        labels = kpts.type(torch.FloatTensor)
        outputs = model(inputs)
        outputs = outputs.type(torch.FloatTensor)
        outputs.to(device)
        test_mse += criterion1(outputs,labels).item() * inputs.size(0)

        outputs = outputs.detach().numpy()
        labels = labels.detach().numpy()
        
        mpjpe, pampjpe = criterion2(outputs,labels)
        test_mpjpe += mpjpe.item() * inputs.size(0)
        test_pampjpe += pampjpe.item() * inputs.size(0)
    test_mpjpe = test_mpjpe/len(tensor_loader.dataset)
    test_pampjpe = test_pampjpe/len(tensor_loader.dataset)
    test_mse = test_mse/len(tensor_loader.dataset)
    print("mse: {:.8f}, mpjpe: {:.8f}, pampjpe: {:.8f}".format(float(test_mse), float(test_mpjpe),float(test_pampjpe)))
    return test_mpjpe

In [ ]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
test(model, val_loader, train_criterion, test_criterion, device)

In [23]:
def train(model, train_loader, test_loader, num_epochs, learning_rate, train_criterion, test_criterion, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
    # optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[20,40],gamma=0.1)
    parameter_dir = './CSI_benchmark/posenet.pt'
    best_test_mpjpe = 100
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        epoch_accuracy = 0
        for data in tqdm(train_loader):
            depth_data, kpts = data
            inputs = depth_data.to(device).unsqueeze(1)
            labels = kpts.to(device)
            labels = labels.type(torch.FloatTensor)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.to(device)
            outputs = outputs.type(torch.FloatTensor)
            # print(outputs.shape, labels.shape)
            loss = train_criterion(outputs,labels)
            loss.backward()
            # print(length)
            # print("loss is ", loss.item())
            optimizer.step()
            
            epoch_loss += loss.item() * inputs.size(0)
            # print("epoch loss is ", epoch_loss)
        epoch_loss = epoch_loss/len(train_loader.dataset)
        print('Epoch: {}, Loss: {:.8f}'.format(epoch, epoch_loss))
        if (epoch+1) % 5 == 0:
            test_mpjpe = test(
                model=model,
                tensor_loader=test_loader,
                criterion1 = train_criterion,
                criterion2 = test_criterion,
                device= device
            )
            if test_mpjpe <= best_test_mpjpe:
                print(f"best test mpjpe is:{test_mpjpe}")
                best_test_mpjpe = test_mpjpe
                torch.save(model.state_dict(), parameter_dir)
        # scheduler.step()
    return

In [24]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) 

cuda:0


In [25]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.load_state_dict(torch.load('./pre-train_weights/lidar_p1_random.pt'))
model.to(device)
train(
    model=model, 
    train_loader= train_loader,
    test_loader= val_loader,    
    num_epochs= 50,
    learning_rate=1e-3,
    train_criterion = train_criterion,
    test_criterion = test_criterion,
    device=device 
    )

100%|██████████| 8019/8019 [2:08:24<00:00,  1.04it/s]  


Epoch: 0, Loss: nan


100%|██████████| 8019/8019 [1:00:17<00:00,  2.22it/s]


Epoch: 1, Loss: nan


100%|██████████| 8019/8019 [27:14<00:00,  4.91it/s]  


...

Epoch: 3, Loss: nan


100%|██████████| 8019/8019 [04:32<00:00, 29.40it/s] 


Epoch: 4, Loss: nan


  0%|          | 0/2005 [00:00<?, ?it/s]


LinAlgError: SVD did not converge